In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from jupyterthemes import jtplot
from datetime import timedelta, datetime
import plotly.express as px
jtplot.style()

# read in fullsample.csv

In [3]:
fs = pd.read_csv('../data/fullsample.csv')

In [9]:
fs = fs[(fs['BEGIN'] != 'Unknown') & (fs['END'] != 'Unknown')]

In [13]:
fs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7395880,25493434,COMPLETED,2020-10-31 23:39:00,2020-10-31 23:40:46,2000Mn,0.09M,2-00:00:00,00:01:46,1,1,sam,0:0
7395881,25493435,COMPLETED,2020-10-31 23:39:13,2020-10-31 23:40:38,2000Mn,187.92M,2-00:00:00,00:01:25,1,1,sam,0:0
7395882,25493476,COMPLETED,2020-10-31 23:46:29,2020-10-31 23:49:43,4096Mc,803.97M,12:00:00,00:03:14,1,1,production,0:0
7395883,25493515,COMPLETED,2020-10-31 23:49:44,2020-10-31 23:51:40,2000Mn,0.09M,2-00:00:00,00:01:56,1,1,sam,0:0


# Get fs info()

In [10]:
fs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7395234 entries, 1 to 7395884
Data columns (total 12 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   STATE      object
 2   BEGIN      object
 3   END        object
 4   REQMEM     object
 5   USEDMEM    object
 6   REQTIME    object
 7   USEDTIME   object
 8   NODES      int64 
 9   CPUS       int64 
 10  PARTITION  object
 11  EXITCODE   object
dtypes: int64(2), object(10)
memory usage: 733.5+ MB


# convert appropriate columns to datetime

In [11]:
dt_converts = ['BEGIN', 'END']

for col in dt_converts:
    fs[col] = pd.to_datetime(fs[col])

<ipython-input-11-f2e450a03f1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs[col] = pd.to_datetime(fs[col])


# filter down to COMPLETED state with 0:0

In [27]:
# one instance of 9204 cancallation

fs.loc[fs['STATE'] == 'CANCELLED by 9204', 'EXITCODE'].value_counts()

1:0    1
Name: EXITCODE, dtype: int64

In [29]:
# all COMPLETED states are exit code 0:0

fs.loc[fs['STATE'] == 'COMPLETED', 'EXITCODE'].value_counts()

0:0    7375084
Name: EXITCODE, dtype: int64

In [30]:
# filter down to only COMPLETED state

fscomp = fs[fs['STATE'] == 'COMPLETED']